In [107]:
import unicodecsv as csv

# Data Analysis Process

'1a: find data'

### Step 1: Get data

Categorical Data - Any value that if you ran mathematical operation on it, you'd get nonsensical data

Ordinal Data - Any value that if you ran mathematical operation on it, you'd get sensical data

In [108]:
#to bring in CSV data sets
#Long way
# enrollment_data = []
# file = open('enrollments.csv', 'rb')

# data=csv.DictReader(file)
# for i in data:
#     enrollment_data.append(i)

# file.close()
# print(enrollment_data)

#Short way
# with open('enrollments.csv', 'rb') as file:
#     data = csv.DictReader(file)
#     enrollment_data = list(data)
# print(enrollment_data)

#Best way
def read_from_csv(filename):
    with open(filename, 'rb') as file:
        data = csv.DictReader(file)
        return list(data)

enrollment_data = read_from_csv('enrollments.csv')
engagement_data = read_from_csv('engagements.csv')
submission_data = read_from_csv('submissions.csv')
# print(enrollment_data[0])
# print(engagement_data[0])
# print(submission_data[0])

### Step 2: Clean Data

In [109]:
from datetime import datetime as dt

def convert_to_date(date_string):
    if date_string == '':
        return None
    return dt.strptime(date_string, '%Y-%m-%d')

def convert_to_int(int_string):
    if int_string == '':
        return None
    return int(int_string)

def convert_to_float(float_string):
    if float_string == '':
        return None
    return float(float_string)

def convert_to_bool(bool_string):
    if bool_string == '':
        return None
    elif bool_string == 'TRUE':
        return True
    elif bool_string == 'FALSE':
        return False



In [110]:
for r in enrollment_data:
    r['join_date'] = convert_to_date(r['join_date'])
    r['cancel_date'] = convert_to_date(r['cancel_date'])
    r['days_to_cancel'] = convert_to_int(r['days_to_cancel'])
    r['is_enrolled'] = convert_to_bool(r['is_enrolled'])
    r['is_canceled'] = convert_to_bool(r['is_canceled'])

In [14]:
enrollment_data[0]

{'account_key': '700',
 'status': 'canceled',
 'join_date': datetime.datetime(2017, 11, 10, 0, 0),
 'cancel_date': datetime.datetime(2017, 11, 16, 0, 0),
 'days_to_cancel': 6,
 'is_enrolled': False,
 'is_canceled': True}

In [111]:
for r in engagement_data:
    r['utc_date'] = convert_to_date(r['utc_date'])
    r['num_courses_visited'] = convert_to_int(r['num_courses_visited'])
    r['total_minutes_visited'] = convert_to_float(r['total_minutes_visited'])
    r['lessons_completed'] = convert_to_int(r['lessons_completed'])
    r['projects_completed'] = convert_to_int(r['projects_completed'])

In [23]:
engagement_data[0]

{'utc_date': datetime.datetime(2018, 1, 9, 0, 0),
 'num_courses_visited': 1,
 'total_minutes_visited': 11.6793745,
 'lessons_completed': 0,
 'projects_completed': 0,
 'account_key': '0'}

In [112]:
for r in submission_data:
    r['creation_date'] = convert_to_date(r['creation_date'])
    r['completion_date'] = convert_to_date(r['completion_date'])

In [18]:
submission_data[0]

{'creation_date': datetime.datetime(2018, 1, 14, 0, 0),
 'completion_date': datetime.datetime(2018, 1, 16, 0, 0),
 'assigned_rating': 'UNGRADED',
 'account_key': '256',
 'lesson_key': '3176718735',
 'processing_state': 'EVALUATED'}

In [113]:
# Column name in engagement_data for 'acct' is not consistent with 'account_key' in other datasets. Change it so it matches 'account_key'
#RUN ONLY ONCE!!!!!!

for row in engagement_data:
    row['account_key'] = row['acct']
    del row['acct']

### Step 3: Answer Questions with Data

In [114]:
print(f"Enrollment records: {len(enrollment_data)}")
print(f"Engagment records: {len(engagement_data)}")
print(f"Submission records: {len(submission_data)}")

Enrollment records: 1640
Engagment records: 136240
Submission records: 3642


In [115]:
def get_unique_records(dataset, column_name):
    unique_data = set()
    for i in dataset:
        unique_data.add(i[column_name])
    return unique_data

unique_enrollments = get_unique_records(enrollment_data, 'account_key')
unique_engagements = get_unique_records(engagement_data, 'account_key')
unique_submissions = get_unique_records(submission_data, 'account_key')

In [116]:
print(f"Unique records: {len(unique_enrollments)}")
print(f"Unique records: {len(unique_engagements)}")
print(f"Unique records: {len(unique_submissions)}")

Unique records: 1302
Unique records: 1237
Unique records: 743


In [117]:
outliers = 0
for i in enrollment_data:
    if i['account_key'] not in unique_engagements and i['join_date'] != i['cancel_date']:
        outliers+=1
        print(i, end='\n\n')

{'account_key': '1304', 'status': 'canceled', 'join_date': datetime.datetime(2018, 1, 10, 0, 0), 'cancel_date': datetime.datetime(2018, 3, 10, 0, 0), 'days_to_cancel': 59, 'is_enrolled': True, 'is_canceled': True}

{'account_key': '1304', 'status': 'canceled', 'join_date': datetime.datetime(2018, 3, 10, 0, 0), 'cancel_date': datetime.datetime(2018, 6, 17, 0, 0), 'days_to_cancel': 99, 'is_enrolled': True, 'is_canceled': True}

{'account_key': '1101', 'status': 'current', 'join_date': datetime.datetime(2018, 2, 25, 0, 0), 'cancel_date': None, 'days_to_cancel': None, 'is_enrolled': True, 'is_canceled': False}



In [118]:
test_accounts = set()
for i in enrollment_data:
    if i['is_enrolled']:
        test_accounts.add(i['account_key'])
print(f"{len(test_accounts)} possible test accounts found")

6 possible test accounts found


In [119]:
#now that we have access to all of the test accounts, we can remove those records from each dataset by it's account_key
def remove_test_accounts(dataset):
    records_to_remove = []
    for i in dataset:
        if i['account_key'] not in test_accounts:
            records_to_remove.append(i)
    return records_to_remove
# this function removes the test accounts from the datasets

In [120]:
true_enrollments = remove_test_accounts(enrollment_data)
true_engagements = remove_test_accounts(engagement_data)
true_submissions = remove_test_accounts(submission_data)

In [96]:
print(f"True enrollments: {len(true_enrollments)}")
print(f"True engagements: {len(true_engagements)}")
print(f"True submissions: {len(true_submissions)}")

True enrollments: 1622
True engagements: 135656
True submissions: 3634


<h4>Find quality accounts in which:</h4>
<ol>
    <li>The account is currently active i.e. is_canceled is False</li>
    <li>days_to_cancel is greater than 7 i.e. the trial period</li>
    <li>The latest vald record for each particular VALID user</li>
</ol>

In [121]:
accounts_in_good_standing = dict()

for i in true_enrollments:
    #the accounts that are currently active or 'is_canceled' is False
    if i['days_to_cancel'] is not None and i['cancel_date'] is not None:
        if not i['is_canceled'] or i['days_to_cancel'] > 7:
            #the latest valid record fore each particulare valid User
            if i['account_key'] not in accounts_in_good_standing or i['join_date'] > accounts_in_good_standing[i['account_key']]:
                accounts_in_good_standing[i['account_key']] = i['join_date']
        
            
    

In [122]:
accounts_in_good_standing

{'429': datetime.datetime(2018, 3, 10, 0, 0),
 '60': datetime.datetime(2018, 1, 14, 0, 0),
 '322': datetime.datetime(2018, 2, 12, 0, 0),
 '584': datetime.datetime(2018, 1, 14, 0, 0),
 '458': datetime.datetime(2017, 11, 10, 0, 0),
 '1058': datetime.datetime(2018, 1, 14, 0, 0),
 '45': datetime.datetime(2017, 11, 10, 0, 0),
 '315': datetime.datetime(2017, 11, 10, 0, 0),
 '408': datetime.datetime(2018, 4, 1, 0, 0),
 '51': datetime.datetime(2018, 3, 10, 0, 0),
 '323': datetime.datetime(2017, 11, 10, 0, 0),
 '130': datetime.datetime(2018, 4, 1, 0, 0),
 '550': datetime.datetime(2018, 5, 28, 0, 0),
 '44': datetime.datetime(2017, 11, 10, 0, 0),
 '440': datetime.datetime(2017, 11, 11, 0, 0),
 '57': datetime.datetime(2017, 11, 11, 0, 0),
 '1090': datetime.datetime(2017, 11, 11, 0, 0),
 '541': datetime.datetime(2017, 11, 12, 0, 0),
 '756': datetime.datetime(2017, 11, 15, 0, 0),
 '101': datetime.datetime(2017, 11, 12, 0, 0),
 '800': datetime.datetime(2018, 3, 4, 0, 0),
 '702': datetime.datetime(201

In [102]:
len(accounts_in_good_standing)

445

In [123]:
# How many students were active on the website within the first week?
def remove_free_trials(dataset):
    free_trials_list = []
    for i in dataset:
        if i['account_key'] in accounts_in_good_standing:
            free_trials_list.append(i)
    return free_trials_list

quality_enrollments = remove_free_trials(true_enrollments)
quality_engagements = remove_free_trials(true_engagements)
quality_submissions = remove_free_trials(true_submissions)

In [124]:
print(f"Quality enrollments records: {len(quality_enrollments)}")
print(f"Quality engagements records: {len(quality_engagements)}")
print(f"Quality submissions records: {len(quality_submissions)}")

Quality enrollments records: 719
Quality engagements records: 55675
Quality submissions records: 1110


In [125]:
# Out of all of the students who lasted past the free trial, 
# how many of them were also active on the website within the first week?
def engagement_within_first_week(date_joined, date_of_first_engagement):
    time_difference = date_of_first_engagement - date_joined
    if time_difference.days >= 0 and time_difference.days < 8:
        return True
    
for i in quality_engagements:
    if i['num_courses_visited'] > 0:
        i['has_visited'] = 1
    else:
        i['has_visited'] = 0
        
first_week_engagements = list()

for i in quality_engagements:
    # accounts_in_good_standing['429'] = datetime.datetime(2018, 3, 10, 0, 0)
    if engagement_within_first_week(accounts_in_good_standing[i['account_key']], i['utc_date']):
        first_week_engagements.append(i)

In [137]:
 print(f"{len(first_week_engagements)} engagements within the first week")

3559 engagements within the first week


In [128]:
from collections import defaultdict

def compare(dataset, a_key):
    new_data = defaultdict(list)
    for i in dataset:
        account_key = i[a_key]
        new_data[account_key].append(i)
    return new_data

engagements_grouped_by_account = compare(first_week_engagements, 'account_key')

In [129]:
# How many minutes did each students spend online in the first week?
def sum_records(dataset, column):
    # dataset = engagements_grouped_by_account
    new_data = dict()
    for k, v in dataset.items():
        # k = engagements_grouped_by_account['5']
        # v = engagements_grouped_by_account['5'][{...}, {...}, {...}, ...]
        sum_ = 0
        for record in v:
            sum_+=record[column]
        new_data[k] = sum_
    return new_data

In [131]:
minutes_per_accounts = sum_records(engagements_grouped_by_account, 'total_minutes_visited')
minutes_per_accounts

{'3': 1023.5579746999999,
 '5': 370.82494396,
 '15': 1071.12533757,
 '18': 155.1361575,
 '19': 0.0,
 '24': 943.1881172000001,
 '25': 0.0,
 '31': 247.27557963999996,
 '34': 138.1789523,
 '42': 174.30704300000002,
 '43': 88.822038,
 '44': 731.72366783,
 '45': 979.7454166629999,
 '51': 0.0,
 '52': 3.227523833,
 '53': 43.53851617,
 '57': 750.894147533,
 '60': 0.0,
 '62': 293.3831222,
 '64': 589.58194586,
 '65': 422.27366,
 '67': 1123.02533437,
 '69': 0.0,
 '72': 883.9906950330001,
 '74': 90.48065633,
 '76': 17.658779,
 '78': 6.609209333,
 '79': 416.495983833,
 '85': 500.49217067,
 '88': 106.16380783,
 '90': 4.842514,
 '91': 70.37317134,
 '92': 715.4277596699999,
 '93': 932.7172561699999,
 '97': 494.84799353,
 '98': 531.4097201300001,
 '99': 425.60350947,
 '101': 188.92038351,
 '105': 573.58079867,
 '106': 535.168407333,
 '108': 164.270890837,
 '110': 1403.143966737,
 '111': 169.1595922,
 '113': 139.57379017,
 '119': 103.90436599,
 '123': 0.0,
 '130': 0.0,
 '132': 1428.3456270000001,
 '134'

In [148]:
most_minutes_spent = sorted(minutes_per_accounts.values(), reverse=True)

[2183.9339814,
 1825.6310638700004,
 1768.5227493,
 1603.15193103,
 1589.158705,
 1467.2371881000001,
 1460.3119036700002,
 1428.3456270000001,
 1427.54662863,
 1421.7436063300001,
 1403.143966737,
 1366.5670150370001,
 1360.78769464,
 1360.7287002000003,
 1334.00295304,
 1229.5553959630001,
 1207.3723326629997,
 1198.3499316,
 1197.02408723,
 1158.3549946,
 1138.31770153,
 1123.238648,
 1123.02533437,
 1122.85546837,
 1098.6832576400002,
 1092.578964797,
 1083.22753966,
 1071.12533757,
 1053.7764164300002,
 1023.5579746999999,
 1011.9525945299999,
 1007.8260931,
 990.7801981700002,
 979.7454166629999,
 969.1894240600001,
 967.46286586,
 959.95343761,
 952.1811420700001,
 947.24981713,
 943.1881172000001,
 934.515744067,
 932.7172561699999,
 931.3076662,
 892.60699383,
 883.9906950330001,
 873.26167336,
 869.7483835,
 851.429489267,
 846.66856726,
 843.5311485300001,
 839.243589237,
 832.77642093,
 832.42184273,
 826.3571918700001,
 824.427983767,
 796.0860388,
 793.77720249,
 783.3383

In [134]:
for k, v in minutes_per_accounts.items():
    if v == most_minutes_spent:
        print(engagements_grouped_by_account[k])
        break

[{'utc_date': datetime.datetime(2018, 3, 8, 0, 0), 'num_courses_visited': 2, 'total_minutes_visited': 155.9647843, 'lessons_completed': 1, 'projects_completed': 0, 'account_key': '171', 'has_visited': 1}, {'utc_date': datetime.datetime(2018, 3, 9, 0, 0), 'num_courses_visited': 2, 'total_minutes_visited': 188.4147333, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '171', 'has_visited': 1}, {'utc_date': datetime.datetime(2018, 3, 10, 0, 0), 'num_courses_visited': 1, 'total_minutes_visited': 554.3022598, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '171', 'has_visited': 1}, {'utc_date': datetime.datetime(2018, 3, 11, 0, 0), 'num_courses_visited': 1, 'total_minutes_visited': 365.7518803, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '171', 'has_visited': 1}, {'utc_date': datetime.datetime(2018, 3, 12, 0, 0), 'num_courses_visited': 1, 'total_minutes_visited': 307.2154792, 'lessons_completed': 1, 'projects_completed': 0, 'account_key': '1

In [ ]:
# HOMEWORK:

# Take a look at data and come up with 3 good questions and use some of the methods done in this 
# Jupyter Notebook to successfully answer the questions.


In [139]:
#Question 1: How many students were active after 30 days?
def engagement_after_30_days(date_joined, date_of_first_engagement):
    time_difference = date_of_first_engagement - date_joined
    if time_difference.days >= 0 and time_difference.days > 30:
        return True
    
for i in quality_engagements:
    if i['num_courses_visited'] > 0:
        i['has_visited'] = 1
    else:
        i['has_visited'] = 0
        
after_30_day_engagements = list()

for i in quality_engagements:
    if engagement_after_30_days(accounts_in_good_standing[i['account_key']], i['utc_date']):
        after_30_day_engagements.append(i)



In [141]:
 print(f"{len(after_30_day_engagements)} engagements after 30 days")

34741 engagements after 30 days


In [142]:
#Question 2: How many lessons did each student that was enagaged after 30 days complete?
engagements_grouped_by_account_after_30 = compare(after_30_day_engagements, 'account_key')

In [143]:
courses_completed = sum_records(engagements_grouped_by_account_after_30, 'lessons_completed')

In [144]:
courses_completed

{'3': 27,
 '5': 6,
 '15': 36,
 '18': 3,
 '19': 9,
 '24': 45,
 '25': 17,
 '31': 18,
 '34': 19,
 '42': 8,
 '43': 4,
 '44': 27,
 '45': 28,
 '51': 2,
 '52': 4,
 '53': 20,
 '57': 19,
 '60': 0,
 '62': 30,
 '64': 4,
 '65': 12,
 '67': 4,
 '69': 1,
 '72': 11,
 '76': 21,
 '78': 19,
 '79': 14,
 '85': 36,
 '88': 1,
 '90': 6,
 '91': 29,
 '92': 43,
 '93': 14,
 '97': 20,
 '98': 5,
 '99': 17,
 '101': 16,
 '105': 1,
 '106': 21,
 '110': 33,
 '111': 5,
 '113': 10,
 '119': 7,
 '123': 0,
 '130': 0,
 '132': 28,
 '134': 8,
 '136': 16,
 '138': 9,
 '140': 51,
 '141': 6,
 '142': 30,
 '143': 13,
 '147': 7,
 '150': 39,
 '151': 1,
 '153': 3,
 '157': 5,
 '159': 19,
 '160': 6,
 '161': 44,
 '165': 5,
 '168': 14,
 '169': 29,
 '170': 26,
 '171': 43,
 '174': 5,
 '178': 3,
 '179': 17,
 '180': 4,
 '181': 15,
 '185': 8,
 '187': 19,
 '190': 12,
 '191': 32,
 '193': 43,
 '197': 22,
 '203': 13,
 '208': 14,
 '217': 23,
 '220': 20,
 '221': 4,
 '227': 11,
 '229': 44,
 '243': 4,
 '251': 13,
 '253': 17,
 '256': 0,
 '258': 26,
 '259

In [152]:
#Question 3: Which Student completed the most courses
most_courses_completed = sorted(courses_completed.values(), reverse=True)[0]

In [157]:
for k, v in courses_completed.items():
    if v == most_courses_completed:
        print(engagements_grouped_by_account_after_30[k][0]['account_key'])
        break

140
